In [57]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [58]:
#超参数
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [59]:
#MNIST数据集
train_dataset = dsets.MNIST(root='../data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='../data/', train=False, transform=transforms.ToTensor(), download=True)

In [60]:
#加载数据集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [61]:
#BiRnn 模型构建
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)).cuda()
        c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)).cuda()
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [62]:
rnn = BiRNN(input_size, hidden_size, num_layers, num_classes)
rnn.cuda()

In [63]:
#定义损失函数、优化函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [64]:
#训练网络
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size)).cuda()
        labels = Variable(labels).cuda()
        
        #前向+后向+优化
        optimizer.zero_grad()
        outputs = rnn(images)
        
        #调用优化函数，计算输出值与真实值的差
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/2], Step [100/600], Loss: 0.7959
Epoch [1/2], Step [200/600], Loss: 0.1915
Epoch [1/2], Step [300/600], Loss: 0.3672
Epoch [1/2], Step [400/600], Loss: 0.1696
Epoch [1/2], Step [500/600], Loss: 0.1618
Epoch [1/2], Step [600/600], Loss: 0.0444
Epoch [2/2], Step [100/600], Loss: 0.0653
Epoch [2/2], Step [200/600], Loss: 0.0644
Epoch [2/2], Step [300/600], Loss: 0.0777
Epoch [2/2], Step [400/600], Loss: 0.0483
Epoch [2/2], Step [500/600], Loss: 0.1443
Epoch [2/2], Step [600/600], Loss: 0.0914


In [52]:
#测试网络
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size)).cuda()
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 97 %


In [66]:
#保存模型参数（但不保存模型）
torch.save(rnn.state_dict(), 'rnn.pkl')

In [ ]:
# 保存和加载整个模型
torch.save(model_object, 'model.pkl')
model = torch.load('model.pkl')

In [ ]:
# 仅保存和加载模型参数(推荐使用)
torch.save(model_object.state_dict(), 'params.pkl')
model_object.load_state_dict(torch.load('params.pkl'))